##  SSD : Single Shot Multibox Detection

- SSD speeds up the process by eliminating the need of the region proposal network.
- Detection composes of 2 parts:

    - Extract feature maps.
    - Apply convolution filters to detect objects.





![title](images/ssd.jpeg)

- Making multiple predictions containing boundary boxes and confidence scores is called multibox
- It computes both the location and class scores using small convolution filters. After extracting the feature maps, SSD applies 3 × 3 convolution filters for each cell to make predictions.
- For example, in Conv4_3, we apply four 3 × 3 filters to map 512 input channels to 25 output channels.
- It uses multiple layers (multi-scale feature maps) to detect objects independently.
- SSD uses lower resolution layers to detect larger scale objects.
- SSD adds 6 more auxiliary convolution layers after the VGG16.

![title](images/ssd_arc.jpeg)

- The default boundary boxes are equivalent to anchors in Faster R-CNN.
- 
- In real-life, boundary boxes do not have arbitrary shape and size.
- Conceptually, the ground truth boundary boxes can be partitioned into clusters with each cluster represented by a default boundary box

![title](images/ssd_box_cluster.jpeg)

- The default boxes are pre-selected manually and carefully to cover a wide spectrum of real life objects.
- Instead of using a global coordination for the box location, the boundary box predictions are relative to the default boundary boxes at each cell (∆cx, ∆cy, w, h).
- For each feature map layers, it shares the same set of default boxes centered at the corresponding cell. 
- Default boundary boxes are chosen manually.
- Starting from the left, Conv4_3 detects objects at the smallest scale 0.2 (or 0.1 sometimes) and then increases linearly to the rightmost layer at a scale of 0.9. Combining the scale value with the target aspect ratios, we compute the width and the height of the default boxes. 
- YOLO uses k-means clustering on the training dataset to determine those default boundary boxes


![title](images/ssd_box.jpeg)

- SSD predictions are classified as positive matches or negative matches. SSD only uses positive matches to calculating the cost in the boundary box mismatch.
-  SSD combines multi-scale feature maps and default boundary boxes to detect objects at different scale. The dog below matches one default box (in red) in the 4 × 4 feature map layer, but not any default boxes in the higher resolution 8 × 8 feature map. The cat which is smaller is detected only by the 8 × 8 feature map layer in 2 default boxes (in blue).

![title](images/ssd_detect.png)

- Higher resolution feature maps are responsible for detecting small objects. The first layer for object detection conv4_3 has a spatial dimension of 38 × 38, a pretty large reduction from the input image. Hence, SSD usually performs badly for small objects comparing with other detection methods. If it is a problem, we can mitigate it by using images with higher resolution.
- The localization loss is the mismatch between the ground truth box and the predicted boundary box. SSD only penalizes predictions from positive matches.

![title](images/loc_loss.png)

- The confidence loss is the loss in making a class prediction. For every positive match prediction, we penalize the loss according to the confidence score of the corresponding class. 

![title](images/con_loss.png)

![title](images/final_loss.png)

 - N is the number of positive match and α is the weight for the localization loss.

##  YOLO : You only look once 

- For our discussion, we crop our original photo. YOLO divides the input image into an S×S grid. Each grid cell predicts only one object.
- Each grid cell predicts a fixed number of boundary boxes.


![title](images/yolo_1.jpeg)

- The one-object rule limits how close detected objects can be. For that, YOLO does have some limitations on how close objects can be. 
- For each grid cell,
    - it predicts B boundary boxes and each box has one box confidence score,
    - it detects one object only regardless of the number of boxes B,
    - it predicts C conditional class probabilities 

- Each boundary box contains 5 elements: (x, y, w, h) and a box confidence score.
- The confidence score reflects how likely the box contains an object (objectness) and how accurate is the boundary box
- We normalize the bounding box width w and height h by the image width and height.
- The conditional class probability is the probability that the detected object belongs to a particular class (one probability per category for each cell). So, YOLO’s prediction has a shape of (S, S, B×5 + C) = (7, 7, 2×5 + 20) = (7, 7, 30).
 

![title](images/yolo_arc.jpeg)

- The major concept of YOLO is to build a CNN network to predict a (7, 7, 30) tensor.
- YOLO performs a linear regression using two fully connected layers to make 7×7×2 boundary box predictions.